In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time

In [3]:
website = 'https://snappfood.ir/service/restaurant/city/semnan/near?page=0&superType=1'
path = 'chromedriver-win64/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))
driver.get(website)
time.sleep(5)
choose_address_button = driver.find_element('xpath', ('//button[@class="sc-fFubgz fQCnPg AutoAddressCard__Layout-sc-90emsr-0 egQPmP"]'))
choose_address_button.click()
time.sleep(2)
choose_semnan_button = driver.find_element('xpath', '//p[@class="sc-hKgILt esHHju"]')
choose_semnan_button.click()
time.sleep(2)
choose_semnan = driver.find_element('xpath', '//p[text()="سمنان"]')
choose_semnan.click()
time.sleep(2)
confirm = driver.find_element('xpath', '//button[text()= "تایید"]')
confirm.click()
time.sleep(5)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup (html, "html.parser")
driver.close()
resturant_links = []
box = soup.find('div', class_='sc-citwmv vwzHB')
href_list = box.find_all('a', class_='VendorCard__HtmlLink-sc-6qaz7-4 cdoeYu', href=True)
for link in href_list:
    resturant_links.append(link['href'])

In [230]:
print(f'https://snappfood.ir{resturant_links[54]}') # to check webpages

https://snappfood.ir/restaurant/menu/%D8%A7%D8%BA%D8%B0%DB%8C%D9%87_%D8%AD%D8%B3_%D8%AE%D9%88%D8%A8__%D8%B4%D8%A8%D8%A7%D9%86%D9%87_-r-3kdodo


In [12]:
df = pd.DataFrame(columns=['date', 'comment', 'rate', 'resturant name'])

In [226]:
root = 'https://snappfood.ir'
url = f'{root}{resturant_links[54]}'

driver = webdriver.Chrome(service=Service(path))
driver.get(url)
comments_button = driver.find_element('xpath', '//button[@class="ModalHeader__ActionButton-sc-ick1zw-1 hMDcUg"]')
comments_button.click()
time.sleep(1)
comments_button = driver.find_element('xpath', '//button[@class="sc-fFubgz hUpWbm"]')
comments_button.click()
time.sleep(5)

popup_element = driver.find_element('xpath', '//section[@class="VendorDetail__Wrapper-sc-1nr3673-1 ffZbOf"]')
current_position = 0
while(1):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', popup_element)
    time.sleep(1)
    last_position = current_position
    current_position = driver.execute_script('return arguments[0].scrollTop', popup_element)
    if current_position == last_position:
        break

comments_page_source = driver.page_source
driver.close()
soup = BeautifulSoup(comments_page_source, 'lxml')
comments_div_section = soup.find('div', class_= 'sc-dlfnbm Comments__CommentsList-sc-137qcx9-1 dUormP gcqMQZ')
comments_div_list = comments_div_section.find_all('div', class_= 'sc-dlfnbm Item__Container-sc-1k0s5ty-0 fBrhOL fsLyDQ')
len(comments_div_list)

25

In [227]:
for comment_div in comments_div_list:
    i = len(df.index)
    df.loc[i, 'date'] = comment_div.find('p', class_ = 'sc-hKgILt hmsjTi').text # date
    try: # rate
        df.loc[i, 'rate'] = comment_div.find('p', class_ = 'sc-hKgILt Item__Rate-sc-1k0s5ty-6 hmsjTi jFBLxL').text # rate
    except:
        df.loc[i, 'rate'] = 'not rated'
    df.loc[i, 'comment'] = comment_div.find_all('p', class_ = 'sc-hKgILt hmsjTi')[1].text # comment
    df.loc[i, 'resturant name'] = soup.find('p', class_='sc-hKgILt filZZV').text # resturant name

In [228]:
df

,date,comment,rate,resturant name
0,۲۷ مهر ۱۴۰۲,خیلی خوشمزه بود و به موقع و داغ رسید به دستم، ...,۵,فست فود کلوا
1,۲۷ مهر ۱۴۰۲,مرغ ها خیلی زیاد سرخ شده بودن,۳,فست فود کلوا
2,۲۵ مهر ۱۴۰۲,خیلی خوشمزه بود,۵,فست فود کلوا
3,۲۶ شهریور ۱۴۰۲,کیفیت بالا ولی قیمت نسبت به حجم پیتزا بالا بود,۵,فست فود کلوا
4,۱۱ مهر ۱۴۰۲,سس بندری خیلی مزه خاصی داشت و یجورایی شیرین بو...,۳,فست فود کلوا
...,...,...,...,...
6393,۱۸ بهمن ۱۴۰۱,دنبال چیز خیلییی خاص نباشین همینک این موقع شب ...,۵,اغذیه حس خوب (شبانه)
6394,۱۸ بهمن ۱۴۰۱,کیفیت همه غذاش مزخرف بود,not rated,اغذیه حس خوب (شبانه)
6395,۱۳ اردیبهشت ۱۴۰۲,,not rated,اغذیه حس خوب (شبانه)
6396,۲۸ آذر ۱۴۰۱,خیلی کیفیت نداشت,۳,اغذیه حس خوب (شبانه)


In [240]:
df = df.drop_duplicates()
df

,date,comment,rate,resturant name
0,۲۷ مهر ۱۴۰۲,خیلی خوشمزه بود و به موقع و داغ رسید به دستم، ...,۵,فست فود کلوا
1,۲۷ مهر ۱۴۰۲,مرغ ها خیلی زیاد سرخ شده بودن,۳,فست فود کلوا
2,۲۵ مهر ۱۴۰۲,خیلی خوشمزه بود,۵,فست فود کلوا
3,۲۶ شهریور ۱۴۰۲,کیفیت بالا ولی قیمت نسبت به حجم پیتزا بالا بود,۵,فست فود کلوا
4,۱۱ مهر ۱۴۰۲,سس بندری خیلی مزه خاصی داشت و یجورایی شیرین بو...,۳,فست فود کلوا
...,...,...,...,...
6393,۱۸ بهمن ۱۴۰۱,دنبال چیز خیلییی خاص نباشین همینک این موقع شب ...,۵,اغذیه حس خوب (شبانه)
6394,۱۸ بهمن ۱۴۰۱,کیفیت همه غذاش مزخرف بود,not rated,اغذیه حس خوب (شبانه)
6395,۱۳ اردیبهشت ۱۴۰۲,,not rated,اغذیه حس خوب (شبانه)
6396,۲۸ آذر ۱۴۰۱,خیلی کیفیت نداشت,۳,اغذیه حس خوب (شبانه)


In [248]:
from jdatetime import datetime
def date_converter(persian_date):
    day, month, year = persian_date.split()
    month_dict = {
        'فروردین' : 1,
        'اردیبهشت' : 2,
        'خرداد' : 3,
        'تیر' : 4,
        'مرداد' : 5,
        'شهریور' : 6,
        'مهر' : 7,
        'آبان' : 8,
        'آذر' : 9,
        'دی' : 10,
        'بهمن' : 11,
        'اسفند' : 12,
    }
    month = month_dict[month]
    gregorian_date = datetime(int(year), month, int(day)).togregorian()
    return str(gregorian_date.strftime('%Y-%m-%d'))

df['date'] = df['date'].apply(date_converter)
df

,date,comment,rate,resturant name
0,2023-10-19,خیلی خوشمزه بود و به موقع و داغ رسید به دستم، ...,۵,فست فود کلوا
1,2023-10-19,مرغ ها خیلی زیاد سرخ شده بودن,۳,فست فود کلوا
2,2023-10-17,خیلی خوشمزه بود,۵,فست فود کلوا
3,2023-09-17,کیفیت بالا ولی قیمت نسبت به حجم پیتزا بالا بود,۵,فست فود کلوا
4,2023-10-03,سس بندری خیلی مزه خاصی داشت و یجورایی شیرین بو...,۳,فست فود کلوا
...,...,...,...,...
6393,2023-02-07,دنبال چیز خیلییی خاص نباشین همینک این موقع شب ...,۵,اغذیه حس خوب (شبانه)
6394,2023-02-07,کیفیت همه غذاش مزخرف بود,not rated,اغذیه حس خوب (شبانه)
6395,2023-05-03,,not rated,اغذیه حس خوب (شبانه)
6396,2022-12-19,خیلی کیفیت نداشت,۳,اغذیه حس خوب (شبانه)


In [249]:
df = df.dropna()
df.to_csv('Snappfood-comments.csv', encoding='utf-8-sig', index=False)